<a href="https://colab.research.google.com/github/Dain777666/ESAA_25-2/blob/main/OB_WEEK2_MON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ch.7 앙상블 학습과 랜덤 포레스트

### 7.1 투표 기반 분류기(VotingClassifier)

- 직접 투표: 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것

- 약한 학습기: 랜덤 추측보다 조금 더 높은 성능을 내는 분류기 -> 앙상블을 통해 강한 학습기가 될 수 있음

- 앙상블 방법은 예측기가 서로 독립적일 때 최고의 성능을 발휘함

- 간접 투표: 모든 분류기가 클래스의 확률을 예측할 수 있으면, 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있음
-> 직접 투표보다 성능이 좋음


In [1]:
import warnings
warnings.filterwarnings('ignore')

# import package
import numpy as np
import os

In [2]:
# 5장에서의 moons dataset 불러오기

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [5]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.85
RandomForestClassifier 0.95
SVC 0.95
VotingClassifier 1.0


### 7.2 배깅과 페이스팅

- 앙상블 방법

1. 각기 다른 훈련 알고리즘을 사용하는 것

2. 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 것

- 배깅: 훈련 세트에서 중복을 허용하여 샘플링하는 방식

- 페이스팅: 훈련 세트에서 중복을 허용하지 않고 샘플링하는 방식

- 분류 -> 통계적 최빈값, 회귀 -> 평균을 계산

### 7.2.1 사이킷런의 배깅과 페이스팅

- BaggingClassifier, BaggingRegressor

- n_jobs=-1: 사용가능한 모든 코어 사용

### 7.2.2 oob 평가

- oob(out-of-bag) 샘플: 선택되지 않은 훈련 샘플의 나머지 37%

- 별도의 검증 세트를 사용하지 않고 oob 샘플을 이용하여 평가 가능함

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [7]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9125

In [8]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.95

In [9]:
bag_clf.oob_decision_function_

array([[0.        , 1.        ],
       [0.0960452 , 0.9039548 ],
       [0.00588235, 0.99411765],
       [0.20606061, 0.79393939],
       [0.96296296, 0.03703704],
       [0.56190476, 0.43809524],
       [0.03448276, 0.96551724],
       [0.86666667, 0.13333333],
       [0.        , 1.        ],
       [0.15083799, 0.84916201],
       [0.98369565, 0.01630435],
       [0.10650888, 0.89349112],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.95705521, 0.04294479],
       [0.06842105, 0.93157895],
       [1.        , 0.        ],
       [0.12060302, 0.87939698],
       [0.99378882, 0.00621118],
       [0.01015228, 0.98984772],
       [0.58522727, 0.41477273],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.00480769, 0.99519231],
       [0.04712042, 0.95287958],
       [0.01086957, 0.98913043],
       [0.5027933 , 0.4972067 ],
       [0.        , 1.        ],
       [0.97826087, 0.02173913],
       [0.98795181, 0.01204819],
       [0.

### 7.3 랜덤 패치와 랜덤 서브스페이스

- BaggingClassifier는 특성 샘플링을 지원함

- max_features, bootstrap_features 두 매개변수로 조절됨

- 랜덤 패치 방식: 훈련 특성과 샘플을 모두 샘플링하는 것

- 랜덤 서브스테이스 방식: 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것

### 7.4 랜덤 포레스트

- 랜덤 포레스트: 배깅 방법을 적용한 결정 트리의 앙상블

- 랜덤 포레스트 알고리즘은 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입함 -> 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만듦

### 7.4.1 엑스트라 트리

- 익스트림 랜덤 트리: 보통의 결정 트리처럼 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그중에서 최상의 분할을 선택함

### 7.4.2 특성 중요도

- 사이킷런은 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정함

- 특성을 선택해야 할 때 어떤 특성이 중요한지 빠르게 확인 가능

In [10]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [12]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

In [13]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.10164612463339252
sepal width (cm) 0.02048736124368528
petal length (cm) 0.43938389106137876
petal width (cm) 0.43848262306154345
